# Load the Dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing the Libraries

# Converting TACO Dataset to YOLO Format and Training YOLOv8

## Overview

This notebook converts the TACO (Trash Annotations in Context) dataset to the YOLO format and trains a YOLOv8 model for waste detection. The TACO dataset contains annotated images of litter in diverse real-world environments, making it useful for object detection tasks related to waste classification.

## Steps Involved

### 1. Convert TACO Annotations to YOLO Format

The provided function convert_taco_to_yolo reads the JSON annotations from the TACO dataset and converts them into the YOLO format. This includes:

- Extracting image and annotation details.

- Splitting the dataset into training (80%) and validation (20%) subsets.

- Saving images and their corresponding label files in YOLO format.

### 2. Generate data.yaml

A data.yaml file is created to define the dataset's path, number of classes, and class names. This file is essential for training YOLO models.

### 3. Train YOLOv8

We use the Ultralytics YOLOv8 model (yolov8n.pt) to train on the converted dataset. The model is trained for 50 epochs with an image size of 640x640 pixels.

## Implementation

### Data Conversion

The function convert_taco_to_yolo performs the following:

- Loads the annotations JSON file.

- Creates directories for training and validation images and labels.

- Copies images to respective directories.

- Converts bounding box coordinates from COCO format (x, y, width, height) to YOLO format (x_center, y_center, width, height as fractions of image dimensions).

- Saves the annotations in .txt format with class IDs.

### Training

- The YOLO class from ultralytics is used to train the YOLOv8 model.

- The training runs for 50 epochs using the dataset defined in data.yaml.

- Results, including loss metrics and mAP (mean Average Precision), can be analyzed for performance evaluation.

## Expected Outcome

After training, the YOLOv8 model will be capable of detecting waste in images. The trained weights can be used for inference on new images or deployed on edge devices for real-time waste detection.

## Potential Applications

- Waste management and recycling automation

- Environmental monitoring

- Smart city initiatives for cleaner public spaces

This notebook provides an efficient pipeline for converting the TACO dataset into YOLO format and training a robust object detection model.



In [ ]:
!pip install -U ultralytics
import ultralytics
ultralytics.checks()

import json
import os
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
import json
import os
from sklearn.model_selection import train_test_split
import shutil
import numpy as np
from ultralytics import YOLO

def convert_taco_to_yolo(json_path, image_root_dir, output_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    images = data['images']
    annotations = data['annotations']
    categories = data['categories']

    os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)

    image_ids = [img['id'] for img in images]
    train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)
    image_id_to_filename = {img['id']: img['file_name'] for img in images}

    for img in images:
        img_id = img['id']
        filename = img['file_name']
        img_width = img['width']
        img_height = img['height']

        if img_id in train_ids:
            image_dir = os.path.join(output_dir, 'images/train')
            label_dir = os.path.join(output_dir, 'labels/train')
        else:
            image_dir = os.path.join(output_dir, 'images/val')
            label_dir = os.path.join(output_dir, 'labels/val')

        full_image_path = os.path.join(image_root_dir, filename)
        if not os.path.exists(full_image_path):
            print(f"Warning: Image {full_image_path} not found.")
            continue
        shutil.copy(full_image_path, os.path.join(image_dir, os.path.basename(filename)))

        label_file = os.path.join(label_dir, os.path.basename(filename).replace('.jpg', '.txt').replace('.JPG', '.txt'))
        with open(label_file, 'w') as lf:
            for ann in annotations:
                if ann['image_id'] == img_id:
                    category_id = ann['category_id']
                    bbox = ann['bbox']
                    x_center = (bbox[0] + bbox[2] / 2) / img_width
                    y_center = (bbox[1] + bbox[3] / 2) / img_height
                    width = bbox[2] / img_width
                    height = bbox[3] / img_height

                    line = f"{category_id} {x_center} {y_center} {width} {height}"

                    # Nếu có segmentation, thêm vào YOLOv8-seg format
                    if 'segmentation' in ann and isinstance(ann['segmentation'], list):
                        for seg in ann['segmentation']:
                            seg_points = np.array(seg, dtype=float).reshape(-1, 2)
                            seg_points[:, 0] /= img_width
                            seg_points[:, 1] /= img_height
                            seg_points_flat = " ".join(map(str, seg_points.flatten()))
                            line += f" {seg_points_flat}"
                    lf.write(line + "\n")

# Gọi hàm
output_yolo_dir = '/kaggle/working/taco_yolo'
image_root_dir = '/kaggle/input/tacotrashdataset/data'
convert_taco_to_yolo('/kaggle/input/tacotrashdataset/data/annotations.json', image_root_dir, output_yolo_dir)

# Tạo file data.yaml
with open('/kaggle/working/taco_yolo/data.yaml', 'w') as f:
    f.write(f"train: {os.path.abspath('/kaggle/working/taco_yolo/images/train')}\n")
    f.write(f"val: {os.path.abspath('/kaggle/working/taco_yolo/images/val')}\n")
    f.write("nc: 60\n")
    categories = json.load(open('/kaggle/input/tacotrashdataset/data/annotations.json', 'r'))['categories']
    f.write("names: " + str([cat['name'] for cat in categories]) + "\n")

# Train YOLOv8-seg
model = YOLO('yolov8m-seg.pt')
results = model.train(data='/kaggle/working/taco_yolo/data.yaml', epochs=50, imgsz=640, project='/kaggle/working/runs', name='taco_train',augment=True,)


# Inference on Validation Images

## Run Inference: Use the trained model to make predictions on the validation images.

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the trained model
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

# Directory containing validation images
val_images_dir = '/kaggle/working/taco_yolo/images/val'

# Directory to save the results.
results_dir = '/kaggle/working/inference_results'
os.makedirs(results_dir, exist_ok=True)

# Iterate over validation images
for filename in os.listdir(val_images_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(val_images_dir, filename)
        image = cv2.imread(image_path)

        # Perform inference
        results = model(image)

        # Visualize results
        annotated_image = results[0].plot()

        # Save annotated image
        output_path = os.path.join(results_dir, f'annotated_{filename}')
        cv2.imwrite(output_path, annotated_image)

print(f"Inference results saved to {results_dir}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from ultralytics import YOLO

# Đường dẫn model đã train
model_path = '/kaggle/working/runs/taco_train/weights/best.pt'
model = YOLO(model_path)

# Đánh giá model
metrics = model.val(data='/kaggle/working/taco_yolo/data.yaml')

# In các chỉ số chính
print(f"mAP@0.5-0.95: {metrics.box.map:.4f}")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"Precision: {metrics.results_dict['metrics/precision(B)']:.4f}")
print(f"Recall: {metrics.results_dict['metrics/recall(B)']:.4f}")

# F1-score (tự tính)
precision = metrics.results_dict['metrics/precision(B)']
recall = metrics.results_dict['metrics/recall(B)']
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"F1-Score: {f1_score:.4f}")

# Đọc file kết quả huấn luyện
results_csv = '/kaggle/working/runs/taco_train/results.csv'
df = pd.read_csv(results_csv)

# Biểu đồ Loss
plt.figure(figsize=(10, 5))
plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss")
plt.plot(df["epoch"], df["train/cls_loss"], label="Train Class Loss")
plt.plot(df["epoch"], df["val/box_loss"], label="Val Box Loss")
plt.plot(df["epoch"], df["val/cls_loss"], label="Val Class Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss per Epoch")
plt.legend()
plt.grid(True)
plt.savefig("/kaggle/working/loss_curve.png")
plt.show()

# Biểu đồ Precision & Recall
plt.figure(figsize=(6, 5))
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Precision and Recall over Epochs")
plt.legend()
plt.grid(True)
plt.savefig("/kaggle/working/precision_recall_curve.png")
plt.show()

# Biểu đồ mAP
plt.figure(figsize=(6, 5))
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("mAP over Epochs")
plt.legend()
plt.grid(True)
plt.savefig("/kaggle/working/map_curve.png")
plt.show()

# Confusion matrix (YOLO tự sinh ra)
# Nếu đã có file, chỉ cần hiển thị lại
conf_path = '/kaggle/working/runs/taco_train/confusion_matrix.png'
if os.path.exists(conf_path):
    from PIL import Image
    img = Image.open(conf_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Confusion Matrix (YOLO auto-generated)")
    plt.show()
else:
    model.plot_confusion_matrix(save_dir='/kaggle/working/runs/taco_train/')
    print("Confusion matrix saved in /kaggle/working/runs/taco_train/")


In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the trained model
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

# Path to your test image (replace with your actual path)
image_path = '/kaggle/input/test-data/download.jpeg'  # Or '/kaggle/working/your_image.jpg'

# Load the image
image = cv2.imread(image_path)

# Perform inference
results = model(image)

# Visualize results
annotated_image = results[0].plot()

# Save the annotated image
output_path = '/kaggle/working/annotated_your_image.jpeg' # or annotated_your_image.jpg
cv2.imwrite(output_path, annotated_image)

print(f"Inference results saved to {output_path}")

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the trained model
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

# Path to your test image (replace with your actual path)
image_path = '/kaggle/input/test-data/download (1).jpeg'  # Or '/kaggle/working/your_image.jpg'

# Load the image
image = cv2.imread(image_path)

# Perform inference
results = model(image)

# Visualize results
annotated_image = results[0].plot()

# Save the annotated image
output_path = '/kaggle/working/annotated_your_image1.jpeg' # or annotated_your_image.jpg
cv2.imwrite(output_path, annotated_image)

print(f"Inference results saved to {output_path}")

In [ ]:
!ls /kaggle/working/runs/taco_train/weights